##Starting code

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd '/content/drive/My Drive/hajsy'

/content/drive/My Drive/hajsy


In [ ]:
import plotly.graph_objects as go

import pandas as pd
import numpy as np
import time
import os
import requests
from pprint import pprint
import subprocess
import datetime as dt
import json
import pandas as pd
import tensorflow as tf

from keras import Input, Model
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.layers import Activation, Dense, Dropout, LSTM, Activation, Flatten, Conv1D, Conv2D, MaxPooling1D, concatenate
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Reshape
from tensorflow import reshape, expand_dims, squeeze
from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay
from xgboost import XGBClassifier
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


In [ ]:
instruments = ["ETHUSDT", "ADAUSDT", "SOLUSDT", "DOTUSDT", "LUNAUSDT", "XLMUSDT", "LTCUSDT", "DOGEUSDT", "BTCUSDT"]
instrument = "DOGEUSDT"

# kline_df = pd.read_csv(f"{instrument}_JAN_OCT.csv", index_col=0)
kline_df = pd.read_csv('data_jan_oct.csv', index_col=0) 

In [ ]:
ml_df = pd.DataFrame()
period_len = 60

ml_df[f"t_future"] = kline_df.open.shift(-1)
for i in range(period_len):
  ml_df[f"t_{i}"] = kline_df.open.shift(i)
ml_df

,t_future,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,t_10,t_11,t_12,t_13,t_14,t_15,t_16,t_17,t_18,t_19,t_20,t_21,t_22,t_23,t_24,t_25,t_26,t_27,t_28,t_29,t_30,t_31,t_32,t_33,t_34,t_35,t_36,t_37,t_38,t_39,t_40,t_41,t_42,t_43,t_44,t_45,t_46,t_47,t_48,t_49,t_50,t_51,t_52,t_53,t_54,t_55,t_56,t_57,t_58,t_59
2021-01-01 00:00:00,28961.67,28923.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:01:00,29009.54,28961.67,28923.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:02:00,28989.68,29009.54,28961.67,28923.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:03:00,28982.67,28989.68,29009.54,28961.67,28923.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:04:00,28975.65,28982.67,28989.68,29009.54,28961.67,28923.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-24 20:35:00,60990.29,60973.73,60937.89,60948.56,60942.24,60833.87,60697.43,60686.30,60674.15,60649.21,60641.09,60678.41,60698.52,60659.07,60643.93,60667.67,60630.00,60659.23,60684.98,60725.75,60777.89,60675.88,60674.85,60638.76,60653.36,60650.31,60607.16,60574.51,60590.00,60598.61,60598.17,60579.44,60650.98,60667.94,60701.54,60725.38,60643.09,60729.98,60619.61,60577.58,60494.20,60515.44,60509.98,60489.35,60518.98,60508.40,60459.36,60420.77,60457.71,60454.05,60481.84,60423.50,60366.09,60331.36,60314.99,60314.05,60357.59,60345.02,60296.80,60350.10,60304.56
2021-10-24 20:36:00,60988.67,60990.29,60973.73,60937.89,60948.56,60942.24,60833.87,60697.43,60686.30,60674.15,60649.21,60641.09,60678.41,60698.52,60659.07,60643.93,60667.67,60630.00,60659.23,60684.98,60725.75,60777.89,60675.88,60674.85,60638.76,60653.36,60650.31,60607.16,60574.51,60590.00,60598.61,60598.17,60579.44,60650.98,60667.94,60701.54,60725.38,60643.09,60729.98,60619.61,60577.58,60494.20,60515.44,60509.98,60489.35,60518.98,60508.40,60459.36,60420.77,60457.71,60454.05,60481.84,60423.50,60366.09,60331.36,60314.99,60314.05,60357.59,60345.02,60296.80,60350.10
2021-10-24 20:37:00,60939.90,60988.67,60990.29,60973.73,60937.89,60948.56,60942.24,60833.87,60697.43,60686.30,60674.15,60649.21,60641.09,60678.41,60698.52,60659.07,60643.93,60667.67,60630.00,60659.23,60684.98,60725.75,60777.89,60675.88,60674.85,60638.76,60653.36,60650.31,60607.16,60574.51,60590.00,60598.61,60598.17,60579.44,60650.98,60667.94,60701.54,60725.38,60643.09,60729.98,60619.61,60577.58,60494.20,60515.44,60509.98,60489.35,60518.98,60508.40,60459.36,60420.77,60457.71,60454.05,60481.84,60423.50,60366.09,60331.36,60314.99,60314.05,60357.59,60345.02,60296.80
2021-10-24 20:38:00,60940.00,60939.90,60988.67,60990.29,60973.73,60937.89,60948.56,60942.24,60833.87,60697.43,60686.30,60674.15,60649.21,60641.09,60678.41,60698.52,60659.07,60643.93,60667.67,60630.00,60659.23,60684.98,60725.75,60777.89,60675.88,60674.85,60638.76,60653.36,60650.31,60607.16,60574.51,60590.00,60598.61,60598.17

In [ ]:
ml_df = ml_df.dropna().astype(float)
ml_df

,t_future,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,t_10,t_11,t_12,t_13,t_14,t_15,t_16,t_17,t_18,t_19,t_20,t_21,t_22,t_23,t_24,t_25,t_26,t_27,t_28,t_29,t_30,t_31,t_32,t_33,t_34,t_35,t_36,t_37,t_38,t_39,t_40,t_41,t_42,t_43,t_44,t_45,t_46,t_47,t_48,t_49,t_50,t_51,t_52,t_53,t_54,t_55,t_56,t_57,t_58,t_59
2021-01-01 00:59:00,28995.13,29021.63,29015.94,28967.14,28958.72,28947.10,28966.05,28966.00,28948.92,28951.44,28944.93,28939.60,28938.76,28929.09,28894.50,28930.11,28916.97,28919.99,28913.32,28927.33,28897.97,28901.91,28914.15,28928.70,28931.41,28879.48,28890.00,28872.49,28858.53,28876.31,28836.63,28833.73,28839.58,28844.48,28871.28,28844.88,28835.01,28796.56,28796.23,28757.97,28822.17,28823.21,28800.58,28812.34,28769.77,28752.80,28716.85,28838.69,28824.36,28848.69,28858.94,28900.00,28934.84,28943.88,28937.11,28975.65,28982.67,28989.68,29009.54,28961.67,28923.63
2021-01-01 01:00:00,28987.62,28995.13,29021.63,29015.94,28967.14,28958.72,28947.10,28966.05,28966.00,28948.92,28951.44,28944.93,28939.60,28938.76,28929.09,28894.50,28930.11,28916.97,28919.99,28913.32,28927.33,28897.97,28901.91,28914.15,28928.70,28931.41,28879.48,28890.00,28872.49,28858.53,28876.31,28836.63,28833.73,28839.58,28844.48,28871.28,28844.88,28835.01,28796.56,28796.23,28757.97,28822.17,28823.21,28800.58,28812.34,28769.77,28752.80,28716.85,28838.69,28824.36,28848.69,28858.94,28900.00,28934.84,28943.88,28937.11,28975.65,28982.67,28989.68,29009.54,28961.67
2021-01-01 01:01:00,28972.27,28987.62,28995.13,29021.63,29015.94,28967.14,28958.72,28947.10,28966.05,28966.00,28948.92,28951.44,28944.93,28939.60,28938.76,28929.09,28894.50,28930.11,28916.97,28919.99,28913.32,28927.33,28897.97,28901.91,28914.15,28928.70,28931.41,28879.48,28890.00,28872.49,28858.53,28876.31,28836.63,28833.73,28839.58,28844.48,28871.28,28844.88,28835.01,28796.56,28796.23,28757.97,28822.17,28823.21,28800.58,28812.34,28769.77,28752.80,28716.85,28838.69,28824.36,28848.69,28858.94,28900.00,28934.84,28943.88,28937.11,28975.65,28982.67,28989.68,29009.54
2021-01-01 01:02:00,28998.95,28972.27,28987.62,28995.13,29021.63,29015.94,28967.14,28958.72,28947.10,28966.05,28966.00,28948.92,28951.44,28944.93,28939.60,28938.76,28929.09,28894.50,28930.11,28916.97,28919.99,28913.32,28927.33,28897.97,28901.91,28914.15,28928.70,28931.41,28879.48,28890.00,28872.49,28858.53,28876.31,28836.63,28833.73,28839.58,28844.48,28871.28,28844.88,28835.01,28796.56,28796.23,28757.97,28822.17,28823.21,28800.58,28812.34,28769.77,28752.80,28716.85,28838.69,28824.36,28848.69,28858.94,28900.00,28934.84,28943.88,28937.11,28975.65,28982.67,28989.68
2021-01-01 01:03:00,28988.16,28998.95,28972.27,28987.62,28995.13,29021.63,29015.94,28967.14,28958.72,28947.10,28966.05,28966.00,28948.92,28951.44,28944.93,28939.60,28938.76,28929.09,28894.50,28930.11,28916.97,28919.99,28913.32,28927.33,28897.97,28901.91,28914.15,28928.70,28931.41,28879.48,28890.00,28872.49,28858.53,28876.31,28836.63,28833.73,28839.58,28844.48,28871.28,28844.88,28835.01,28796.56,28796.23,28757.97,28822.17,28823.21,28800.58,28812.34,28769.77,28752.80,28716.85,28838.69,28824.36,28848.69,28858.94,28900.00,28934.84,28943.88,28937.11,28975.65,28982.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-24 20:34:00,60973.73,60937.89,60948.56,60942.24,60833.87,60697.43,60686.30,60674.15,60649.21,60641.09,60678.41,60698.52,60659.07,60643.93,60667.67,60630.00,60659.23,60684.98,60725.75,60777.89,60675.88,60674.85,60638.76,60653.36,60650.31,60607.16,60574.51,60590.00,60598.61,60598.17,60579.44,60650.98,60667.94,60701.54,60725.38,60643.09,60729.98,60619.61,60577.58,60494.20,60515.44,60509.98,60489.35,60518.98,60508.40,60459.36,60420.77,60457.71,60454.05,60481.84,60423.50,60366.09,60331.36,60314.99,60314.05,60357.59,60345.02,60296.80,60350.10,60304.56,60283.85
2021-10-24 20:35:00,60990.29,60973.73,

In [ ]:
time_window = 15 # in minutes, we predict whether the price will raise in next {time_window} minutes
transaction_fee = 0.999
min_gain = 0.001 # in %

last_price = ml_df[f"t_0"]# * (1 + min_gain))
y = (ml_df["t_future"] * transaction_fee) > last_price

# for i in range(time_window):
    # y = y | ((ml_df[f"t_{i}"] * transaction_fee * transaction_fee) > (ml_df[f"t_{time_window}"] * (1 + min_gain)))

x = ml_df.drop(columns=[f't_future'])

xnp = x.to_numpy()
ynp = y.to_numpy()
print(xnp.shape)
print(ynp.shape)

(426413, 60)
(426413,)


## previous


In [ ]:
def strategy_long_binary_classifier(model, state, price, prev_price, have_stock = False,  money = 1000, stock = 0, time_window = 15, time_with_stock = 0, min_gain = 0.001):
    transaction_fee = 0.999
    if have_stock:
        time_with_stock += 1
        if (prev_price * (1+min_gain)) < (price * transaction_fee) or time_with_stock > (time_window*2):
            if time_with_stock > time_window:
                print("SELL STOP LOSS")
            print(f"SELL : PRICE {price} PREV {prev_price}")
            money += stock * price * transaction_fee
            stock = 0
            have_stock = False

    elif np.any(model.predict(state.reshape(1,-1))):
        print(f"BUY : PRICE {price}")
        time_with_stock = 0
        stock = (money / price) * transaction_fee
        prev_price = price
        money = 0
        have_stock = True

    return money, stock, have_stock, prev_price, time_with_stock

def test_strategy_long(df, model, params = [], period_len = 60, time_window = 15, min_gain = 0.001, train_size = 0.8, test_size = 0.2):
    assert(train_size + test_size == 1.0)

    transaction_fee = 0.999
    ml_df = pd.DataFrame()

    for i in range(period_len):
        ml_df[f"t_{i}"] = df.open.shift(i)
    print(ml_df.shape)
    ml_df = ml_df.dropna().astype(float)
    print(ml_df.shape)

    last_prices = (ml_df[f"t_{time_window}"] * (1 + min_gain))
    y = (ml_df["t_0"] * transaction_fee * transaction_fee) > last_prices

    for i in range(time_window):
        y = y | ((ml_df[f"t_{i}"] * transaction_fee * transaction_fee) > (ml_df[f"t_{time_window}"] * (1 + min_gain)))

    ml_df = ml_df.drop(columns=[f"t_{i}" for i in range(time_window)])

    Xnp = ml_df.to_numpy()
    ynp = y.to_numpy()
    X_train, X_test, y_train, y_test = train_test_split(Xnp, ynp, train_size = train_size , test_size=test_size, shuffle = False)
    model.fit(X_train, y_train)

    money = 1000
    stock = 0
    num_trans = 0
    time_with_stock = 0
    have_stock = False
    prev_price = 0
    sells = []
    buys = []
    moneys = []

    for i in range(int(ml_df.shape[0]*train_size), ml_df.shape[0]):
        prev_state = have_stock
        money, stock, have_stock, prev_price, time_with_stock = strategy_long_binary_classifier(model, Xnp[i], Xnp[i][0], prev_price, have_stock, money, stock, time_window, time_with_stock, min_gain)

        if have_stock != prev_state:
            num_trans+=1
            if have_stock:
                buys.append(i)
            else:
                sells.append(i)
                #print(Xnp[i][0])
                #print(ml_df["t_15"][i])
                #print(df[period_len-1:].open[i])
                moneys.append(money)

    return money, stock, num_trans, sells, buys, moneys

def buys_sells_plot(df, buys, sells, period_len = 60, time_window = 15):
    print(df[period_len-1:].shape)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df[period_len-1-time_window:].index, y=df[period_len-1-time_window:].open, mode="lines", name="price"))
    fig.add_trace(go.Scatter(x=df[period_len-1-time_window:].index[buys], y=df[period_len-1-time_window:].open[buys], mode="markers", marker=dict(size=12, color="green"), name='Buy signal'))
    fig.add_trace(go.Scatter(x=df[].index[sells], y=df[period_len-1-time_window:].open[sells], mode="markers", marker=dict(size=12, color="blue"), name='Sell signal'))
    return fig

def money_plot(df, sells, moneys, period_len = 60, time_window = 15):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df[period_len-1:].index[sells], y=moneys, mode="lines", name="price"))
    return fig



In [ ]:
(1007 * (1+min_gain)) < (1002 * transaction_fee * transaction_fee)

False

In [ ]:
instrument = "DOGEUSDT"

kline_df = pd.read_csv(f"{instrument}_JAN_OCT.csv", index_col=0)

money, stock, num_trans, sells, buys, moneys = test_strategy_long(kline_df[50000:100000], XGBClassifier(max_depth =5), min_gain=0.003)
print(sells)
buys_sells_plot(kline_df[50000:100000], buys,sells).show()
print(moneys)


## fixed lstm

In [ ]:

period_len=300
size_std = 4
kline_df["ma"] = kline_df['close'].rolling(period_len).mean() + kline_df['close'].rolling(period_len).std()# 5h hourse
kline_df["std"] = kline_df['close'].rolling(period_len).std()
kline_df["bb_u"] = kline_df["ma"] + size_std*kline_df["std"]
kline_df["bb_l"] = kline_df["ma"] - size_std*kline_df["std"]


In [ ]:

period_len=300
size_std = 4
kline_df2["ma"] = kline_df2['close'].rolling(period_len).mean() + kline_df2['close'].rolling(period_len).std()# 5h hourse
kline_df2["std"] = kline_df2['close'].rolling(period_len).std()
kline_df2["bb_u"] = kline_df2["ma"] + size_std*kline_df2["std"]
kline_df2["bb_l"] = kline_df2["ma"] - size_std*kline_df2["std"]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [ ]:
def process_prices(kline_df, time_window=15, period_len=60, target_col='close'):
  ml_df = pd.DataFrame()
  period_len = period_len
  time_window = time_window # in minutes, we predict whether the price will raise in next {time_window} minutes
  ml_df[f"t_future"] = kline_df[target_col].shift(-time_window)
  for i in range(period_len):
    ml_df[f"t_{i}"] = kline_df[target_col].shift(i)
  # ml_df
  ml_df = ml_df[500:]
  ml_df = ml_df.dropna().astype(float)

  transaction_fee = 0.999
  min_gain = 0.001 # in %

  last_price = ml_df[f"t_0"]# * (1 + min_gain))
  
  y = (ml_df["t_future"] ) > last_price
  x = ml_df.drop(columns=[f't_future'])

  xnp = x.to_numpy()
  ynp = y.to_numpy()
  print(xnp.shape)
  print(ynp.shape)
  # xnp = xnp.reshape(xnp.shape[0], xnp.shape[1], 1)
  return xnp, ynp

In [ ]:
def add_feature(current_X, new_feature, period_len, omit_num=15, kline_df=kline_df):
  ml_df = pd.DataFrame()
  period_len = period_len
  for i in range(period_len):
    ml_df[f"t_{i}"] = kline_df[new_feature].shift(i)
  ml_df
  ml_df = ml_df.dropna().astype(float)
  ml_df = ml_df[:-omit_num]
  xnp = ml_df.to_numpy()
  
  print(xnp.shape)
  xnp = xnp[-current_X.shape[0]:]
  
  return np.dstack((current_X, xnp))

In [ ]:
kline_df

,open,high,low,close,vol
2021-01-01 00:00:00,28923.63,28961.66,28913.12,28961.66,27.457032
2021-01-01 00:01:00,28961.67,29009.91,28961.01,29017.50,58.477501
2021-01-01 00:02:00,29009.54,28989.30,28973.58,29016.71,42.470329
2021-01-01 00:03:00,28989.68,28982.69,28972.33,28999.85,30.360677
2021-01-01 00:04:00,28982.67,28975.65,28971.80,28995.93,24.124339
...,...,...,...,...,...
2021-10-24 20:35:00,60973.73,60990.30,60942.99,61010.79,26.955470
2021-10-24 20:36:00,60990.29,60988.67,60967.21,60996.24,15.450520
2021-10-24 20:37:00,60988.67,60939.90,60939.90,60988.67,39.582950
2021-10-24 20:38:00,60939.90,60939.99,60928.71,60950.00,11.011360


In [ ]:
kline_df2 = kline_df.iloc[::5,:]
kline_df2

,open,high,low,close,vol
2021-01-01 00:00:00,28923.63,28961.66,28913.12,28961.66,27.457032
2021-01-01 00:05:00,28975.65,28937.11,28933.16,28979.53,22.396014
2021-01-01 00:10:00,28858.94,28848.68,28848.06,28883.20,42.665900
2021-01-01 00:15:00,28752.80,28769.77,28720.91,28775.99,45.695385
2021-01-01 00:20:00,28822.17,28759.35,28751.58,28822.17,35.799104
...,...,...,...,...,...
2021-10-24 20:18:00,60684.98,60659.23,60650.00,60700.01,10.283760
2021-10-24 20:23:00,60659.07,60698.53,60640.90,60700.00,35.601640
2021-10-24 20:28:00,60674.15,60686.30,60667.06,60692.50,10.890930
2021-10-24 20:33:00,60948.56,60942.13,60930.05,60998.85,32.688860


In [ ]:
X, Y = process_prices(kline_df, time_window=1, period_len=30)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


(425986, 30)
(425986,)


In [ ]:
X = X.reshape((X.shape[0], X.shape[1],1))

In [ ]:
X3 = add_feature(X, 'high', period_len=30, kline_df=kline_df)
X3 = add_feature(X3, 'low', period_len=30, kline_df=kline_df)
X3 = add_feature(X3, 'close', period_len=30, kline_df=kline_df)
X3 = add_feature(X3, 'vol', period_len=30, kline_df=kline_df)


(426443, 30)
(426443, 30)
(426443, 30)
(426443, 30)


In [ ]:
X, Y = process_prices(kline_df2, time_window=3, period_len=60)

In [ ]:
X3.shape

(425986, 30, 5)

In [ ]:
def test_strategy_long_nn(Xnp, ynp, params = [], period_len = 60, time_window = 15, min_gain = 0.001, train_size = 0.8, test_size = 0.2):
    assert(train_size + test_size == 1.0)
    assert(Xnp.shape[0] == ynp.shape[0])

    # transaction_fee = 0.999
    # ml_df = pd.DataFrame()

    # for i in range(period_len):
    #     ml_df[f"t_{i}"] = df.open.shift(i)
    # print(ml_df.shape)
    # ml_df = ml_df.dropna().astype(float)
    # print(ml_df.shape)

    # last_prices = (ml_df[f"t_{time_window}"] * (1 + min_gain))
    # y = (ml_df["t_0"] * transaction_fee * transaction_fee) > last_prices

    # for i in range(time_window):
    #     y = y | ((ml_df[f"t_{i}"] * transaction_fee * transaction_fee) > (ml_df[f"t_{time_window}"] * (1 + min_gain)))

    # ml_df = ml_df.drop(columns=[f"t_{i}" for i in range(time_window)])

    # Xnp = ml_df.to_numpy()
    # ynp = y.to_numpy()
    #ynp = to_categorical(ynp)
    
    ynp = ynp.astype(int)

    X_train, X_test, y_train, y_test = train_test_split(Xnp, ynp, train_size = train_size , test_size=test_size, shuffle = False)
    
    print(X_train.shape)
    print(f'fraction is {sum(y_train.astype(int)) / y_train.shape[0]}')

    for i in range(X_train.shape[2]):
      sc_train = MinMaxScaler(feature_range = (0, 1))
      sc_test = MinMaxScaler(feature_range = (0, 1))
      X_train[...,i] = sc_train.fit_transform(X_train[...,i])
      X_test[...,i] = sc_test.fit_transform(X_test[...,i])
      # X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
      # X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))
    model = LSTMBasic(X_train)
    train_model(model, X_train, X_test, y_train, y_test)
    return model, X_train, X_test, y_train, y_test

    # money = 1000
    # stock = 0
    # num_trans = 0
    # time_with_stock = 0
    # have_stock = False
    # prev_price = 0
    # sells = []
    # buys = []
    # moneys = []

    # for i in range(int(ml_df.shape[0]*train_size), ml_df.shape[0]):
    #     prev_state = have_stock
    #     money, stock, have_stock, prev_price, time_with_stock = strategy_long_binary_classifier(model, Xnp[i], Xnp[i][0], prev_price, have_stock, money, stock, time_window, time_with_stock, min_gain)

    #     if have_stock != prev_state:
    #         num_trans+=1
    #         if have_stock:
    #             buys.append(i)
    #         else:
    #             sells.append(i)
    #             moneys.append(money)

    # return money, stock, num_trans, sells, buys, moneys

def train_model(model, X_train, X_test, y_train, y_test):
    print(X_train.shape)
    print(X_test.shape)
    print(y_train.shape)
    print(y_test.shape)

    with tf.device('/device:GPU:0'):
        model.fit(X_train, y_train, epochs = 300, batch_size = 64)
    
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))

    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    print(sum(y_pred))
    # (model.predict(x) > 0.5).astype("int32")
    #print(y_pred[:10])
    print(y_pred.shape)
    print(y_test.shape)
    

def LSTMBasic(X_train):
    
    model = Sequential()

    # model.add(LSTM(units = 100, return_sequences=True, input_shape = (X_train.shape[1], X_train.shape[2])))
    # model.add(Dropout(0.2))
    # model.add(LSTM(units = 100, return_sequences=True))
    # model.add(Dropout(0.2))
    # model.add(LSTM(units = 100, return_sequences=True))
    # model.add(Dropout(0.2))
    # model.add(LSTM(units = 100, return_sequences=False))
    # # model.add(Dropout(0.2))
    # model.add(Dense(units = 200, activation = "relu"))
    # model.add(Dense(units = 50, activation = "relu"))
    # model.add(Dense(units = 1, activation = "sigmoid"))

    model.add(LSTM(units = 50, return_sequences=True, input_shape = (X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(units = 50, return_sequences=False))
    model.add(Dense(units = 1, activation = "sigmoid"))
    sgd = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

    model.compile(loss = 'binary_crossentropy',  metrics=['accuracy'], optimizer=sgd)
    print(model.summary())

    return model



In [ ]:
fraction = 10000
print(sum(Y[-fraction:].astype(int)) / Y[-fraction:].shape[0])

0.4887


In [ ]:
kline_df

,open,high,low,close,vol
2021-01-01 00:00:00,28923.63,28961.66,28913.12,28961.66,27.457032
2021-01-01 00:01:00,28961.67,29009.91,28961.01,29017.50,58.477501
2021-01-01 00:02:00,29009.54,28989.30,28973.58,29016.71,42.470329
2021-01-01 00:03:00,28989.68,28982.69,28972.33,28999.85,30.360677
2021-01-01 00:04:00,28982.67,28975.65,28971.80,28995.93,24.124339
...,...,...,...,...,...
2021-10-24 20:35:00,60973.73,60990.30,60942.99,61010.79,26.955470
2021-10-24 20:36:00,60990.29,60988.67,60967.21,60996.24,15.450520
2021-10-24 20:37:00,60988.67,60939.90,60939.90,60988.67,39.582950
2021-10-24 20:38:00,60939.90,60939.99,60928.71,60950.00,11.011360


In [ ]:
X_test

array([[66426.12, 66440.3 , 66418.46, ..., 66047.25, 65999.99, 65936.7 ],
       [66401.08, 66426.12, 66440.3 , ..., 66026.44, 66047.25, 65999.99],
       [66425.12, 66401.08, 66426.12, ..., 65998.46, 66026.44, 66047.25],
       ...,
       [60990.29, 60973.73, 60937.89, ..., 60131.81, 60133.15, 60194.  ],
       [60988.67, 60990.29, 60973.73, ..., 60149.89, 60131.81, 60133.15],
       [60939.9 , 60988.67, 60990.29, ..., 60137.36, 60149.89, 60131.81]])

In [ ]:
sum(y_test)

553

## first successful training, simple LSTM network, 
~40 days training, ~15 testing <br>
30 previous minutes <br>
5 features - open, close, volume, high, low <br>
predicts one minute in the future only

In [ ]:
model, X_train, X_test, y_train, y_test = test_strategy_long_nn(X3[-80000:], Y[-80000:], test_size = 0.3, train_size=0.7)


(56000, 30, 5)
fraction is 0.4559285714285714


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 50)            11200     
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 31,451
Trainable params: 31,451
Non-trainable params: 0
_________________________________________________________________
None
(56000, 30, 5)
(24000, 30, 5)
(56000,)
(24000,)
Epoch 1/300
875/875 [==============================] - 23s 23ms/step - loss: 0.6898 - accuracy: 0.5432
Epoch 2/300
875/875 [==============================] - 20s 23ms/step - loss: 0.6894 - accuracy: 0.5441
Epoch 3/300
875/875 [==============================] - 20

In [ ]:
model.save('model_300ep_5feature30min')

INFO:tensorflow:Assets written to: model_300ep_5feature30min/assets


INFO:tensorflow:Assets written to: model_300ep_5feature30min/assets


## results 1 network 

### proportion of ups

In [ ]:
sum(y_test) / 24000

0.4535416666666667

In [ ]:
prediction = model.predict(X_test)

In [ ]:
prediction.shape

(24000, 1)

In [ ]:
where1 = np.where(prediction < 0.5)[0]
print(where1.shape)
print(sum(y_test[where1]))

(19051,)
8578


In [ ]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")
sum((y_pred.reshape(y_pred.shape[0]) == y_test).astype(int)) / 24000 #prediction quality full

0.53475

### correct predictions - up
higher then proportion of up in data, which is 0.4535


In [ ]:
print(sum(y_test) / 24000)

prediction = model.predict(X_test)
where1 = np.where(prediction >= 0.50)[0]
print(where1.shape)
print(sum(y_test[where1]) / where1.shape) #quality up, 


0.4535416666666667
(4437,)
[0.46833446]


### correct predictions - down
higher then proportion of down in data, which is 0.5465

In [ ]:
print(sum(y_test) / 24000)

prediction = model.predict(X_test)
where1 = np.where(prediction <= 0.50)[0]
print(where1.shape)
print((where1.shape[0] - sum(y_test[where1]) )/ where1.shape) #quality down, higher then proportion of down in data


0.4535416666666667
(19563,)
[0.54981342]


## second try
~40 days training, ~15 testing <br>
60 previous minutes <br>
5 features - open, close, volume, high, low <br>
now predicts 15 minutes into the future - will go up or noy

In [ ]:
X, Y = process_prices(kline_df, time_window=15, period_len=60)

X3 = add_feature(X, 'high', period_len=60, kline_df=kline_df)
X3 = add_feature(X3, 'low', period_len=60, kline_df=kline_df)
X3 = add_feature(X3, 'close', period_len=60, kline_df=kline_df)
X3 = add_feature(X3, 'vol', period_len=60, kline_df=kline_df)


(425972, 60)
(425972,)
(426413, 60)
(426413, 60)
(426413, 60)
(426413, 60)


In [ ]:
model2, X_train2, X_test2, y_train2, y_test2 = test_strategy_long_nn(X3[-80000:], Y[-80000:], test_size = 0.3, train_size=0.7)


(56000, 60, 5)
fraction is 0.4961607142857143


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 60, 50)            11200     
                                                                 
 dropout_1 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_5 (LSTM)               (None, 50)                20200     
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 31,451
Trainable params: 31,451
Non-trainable params: 0
_________________________________________________________________
None
(56000, 60, 5)
(24000, 60, 5)
(56000,)
(24000,)
Epoch 1/300
875/875 [==============================] - 41s 44ms/step - loss: 0.6933 - accuracy: 0.5031
Epoch 2/300
875/875 [=====

In [ ]:
model2.save('model_300ep_5feature60min_plus15')

INFO:tensorflow:Assets written to: model_300ep_5feature60min_plus15/assets


INFO:tensorflow:Assets written to: model_300ep_5feature60min_plus15/assets


## resuts second network

In [ ]:
print(sum(y_test2) / 24000) # proportion

0.493


total accuracy

In [ ]:
y_pred2 = (model2.predict(X_test2) > 0.5).astype("int32")
sum((y_pred2.reshape(y_pred2.shape[0]) == y_test2).astype(int)) / 24000 #prediction quality

0.5139583333333333

### correct prediction - up
much better then random, up from 0.493 proportion in data

In [ ]:
print(sum(y_test2) / 24000)
prediction = model2.predict(X_test2)
where1 = np.where(prediction >= 0.50)[0]
print(where1.shape)
print(sum(y_test2[where1]) / where1.shape) #quality up


0.493
(4949,)
[0.5168721]


### correct prediction - down
also better then random, up from 0.507 proportion in data

In [ ]:
print(sum(y_test2) / 24000)
prediction = model2.predict(X_test2)
where1 = np.where(prediction <= 0.50)[0]
print(where1.shape)
print((where1.shape[0] - sum(y_test2[where1]) )/ where1.shape) #quality down


0.493
(19051,)
[0.51320141]


In [ ]:
prediction

array([[0.55799615],
       [0.5586262 ],
       [0.5592814 ],
       ...,
       [0.4868262 ],
       [0.48685765],
       [0.4868237 ]], dtype=float32)